# Stable Diffusion WebUI (AUTOMATIC1111) for SageMaker Studio

### Installation: Clone webui repository

In [ ]:
%cd ~/notebooks/
!git clone --depth 1 https://github.com/AUTOMATIC1111/stable-diffusion-webui
!ln -vs /tmp/outputs stable-diffusion-webui/outputs
!ln -vs /tmp ~/notebooks/tmp
!ln -vs /tmp/models stable-diffusion-webui/models/Stable-diffusion/tmp_models

# install extensions (not necessary)
%cd stable-diffusion-webui/extensions
!git clone --depth 1 https://github.com/yfszzx/stable-diffusion-webui-images-browser
!git clone --depth 1 https://github.com/toshiaki1729/stable-diffusion-webui-dataset-tag-editor

### Installation: Download model(s) from Huggingface

https://huggingface.co/CompVis/stable-diffusion-v-1-4-original

https://huggingface.co/runwayml/stable-diffusion-v1-5

https://huggingface.co/runwayml/stable-diffusion-inpainting

In [ ]:
# import ipywidgets as widgets

# token_textbox = widgets.Text(
#     value='HUGGING_FACE_TOKEN',
#     description='Token:',
# )
# token_textbox

In [ ]:
huggingface_token = ""

# import os

# token_path = os.path.expanduser('~/.huggingface/token')

# if os.path.isfile(token_path):
#     with open(token_path) as f:
#         lines = f.readlines()
#     huggingface_token = lines[0]
# else:
#     try:
#         token_textbox
#     except NameError:
#         raise RuntimeError("Enter Huggingface token")
#     else:
#         !mkdir -p ~/.huggingface
#         !echo -n "{token_textbox.value}" > ~/.huggingface/token
#         huggingface_token = token_textbox.value

user_header = f"\"Authorization: Bearer {huggingface_token}\""

%cd ~/notebooks/stable-diffusion-webui/models/Stable-diffusion/

# Model 1.4
!wget --header={user_header} https://huggingface.co/CompVis/stable-diffusion-v-1-4-original/resolve/main/sd-v1-4.ckpt

# Model 1.5 (8GB) - only necessary for native training, not for embeddings/hypernetworks
!wget --header={user_header} https://huggingface.co/runwayml/stable-diffusion-v1-5/resolve/main/v1-5-pruned.ckpt 

# Model 1.5 (4GB)
!wget --header={user_header} https://huggingface.co/runwayml/stable-diffusion-v1-5/resolve/main/v1-5-pruned-emaonly.ckpt 

# Inpainting model 1.5 (4GB)
!wget --header={user_header} https://huggingface.co/runwayml/stable-diffusion-inpainting/resolve/main/sd-v1-5-inpainting.ckpt

### Recommended Optional: Download variational autoencoder (VAE)

https://huggingface.co/stabilityai/sd-vae-ft-ema-original

https://huggingface.co/stabilityai/sd-vae-ft-mse-original

In [ ]:
# sd-vae-ft-ema-original
%cd ~/notebooks/stable-diffusion-webui/models/VAE/
!wget --header={user_header} https://huggingface.co/stabilityai/sd-vae-ft-ema-original/resolve/main/vae-ft-ema-560000-ema-pruned.ckpt

# sd-vae-ft-mse-original
%cd ~/notebooks/stable-diffusion-webui/models/VAE/
!wget --header={user_header} https://huggingface.co/stabilityai/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.ckpt

# sd-vae-ft-mse-original
# Use this if you downloaded v1-5-pruned-emaonly.ckpt above and want to autoselect the VAE every time you load this model in the webui
%cd ~/notebooks/stable-diffusion-webui/models/Stable-diffusion/
!wget --header={user_header} https://huggingface.co/stabilityai/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.ckpt -O v1-5-pruned-emaonly.vae.pt

### Optional: Download various models

https://huggingface.co/prompthero/midjourney-v4-diffusion

https://huggingface.co/hakurei/waifu-diffusion-v1-4

In [ ]:
############################################################

download_to_temp_folder = False # models will be deleted when the SageMaker session ends 

midjourney_v4 = True # Midjourney V4 Diffusion
waifu = True # Waifu Division with Anime VAE

############################################################

if download_to_temp_folder:
    !mkdir -p /tmp/models
    %cd ~/notebooks/stable-diffusion-webui/models/Stable-diffusion/tmp_models
else:
    %cd ~/notebooks/stable-diffusion-webui/models/Stable-diffusion

if midjourney_v4:
    !wget https://huggingface.co/prompthero/midjourney-v4-diffusion/resolve/main/mdjrny-v4.ckpt

if waifu:
    %cd ~/notebooks/stable-diffusion-webui/models/Stable-diffusion
    !wget https://huggingface.co/hakurei/waifu-diffusion-v1-4/resolve/main/models/wd-1-3-penultimate-ucg-cont.ckpt

    %cd ~/notebooks/stable-diffusion-webui/models/VAE
    !wget https://huggingface.co/hakurei/waifu-diffusion-v1-4/resolve/main/vae/kl-f8-anime.ckpt
    !wget https://huggingface.co/hakurei/waifu-diffusion-v1-4/resolve/main/vae/kl-f8-anime2.ckpt

### Update: Download updates & Install webui requirements.txt

In [ ]:
!apt update -y && apt upgrade -y

%cd ~/notebooks/stable-diffusion-webui
!git pull

%pip install -U pip setuptools wheel
%pip install -r requirements.txt

In [ ]:
# import ipywidgets as widgets

# ngrok_token_textbox = widgets.Text(
#     value='YOUR_NGROK_TOKEN',
#     description='Token:',
# )
# ngrok_token_textbox

### Run: Launch web ui

In [ ]:
##################################

WEBUI_USERNAME=""
WEBUI_PASSWORD=""

##################################

ngrok_token = ""

# import os
# import sys

# if 'ngrok_token_textbox' in locals():
#     print("Saving ngrok auth token to ~/.ngrok_token")
#     !echo -n "{ngrok_token_textbox.value}" > ~/.ngrok_token
#     ngrok_token = ngrok_token_textbox.value
# else:
#     full_path = os.path.expanduser('~/.ngrok_token')
#     if not os.path.exists(full_path):
#         sys.exit("No ngrok auth token found")
#     else:
#         with open(full_path) as f:
#             ngrok_token = f.readline()
#         print("Using ngrok auth token from ~/.ngrok_token")

!mkdir -p /tmp/outputs
!mkdir -p /tmp/models

%cd ~/notebooks/stable-diffusion-webui
ARGS = "\"--xformers --disable-nan-check --gradio-debug --gradio-auth " + WEBUI_USERNAME + ":" + WEBUI_PASSWORD + " --ngrok " + ngrok_token + "\""
!COMMANDLINE_ARGS=$ARGS REQS_FILE="requirements.txt" python launch.py

### End: Commands for ***after*** you're done with a session
Click the square stop button before running these cells

In [ ]:
%cd /tmp
!echo "Creating archive. Please wait..."
!tar -czf ~/outputs/outputs.tar.gz outputs
!echo "You can now download the archive with generated images"